In [11]:
!pip install missingpy
!pip install xgboost
#Clean up previous tables
import numpy as np
from numpy.random import choice
import pandas as pd
import statistics
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pickle
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import multiprocessing
import seaborn as sns
from matplotlib import pyplot
from sklearn.impute import SimpleImputer
from missingpy import KNNImputer
from missingpy import MissForest
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from collections import Counter
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)

sagemaker 1.38.0 has requirement urllib3<1.25,>=1.21, but you'll have urllib3 1.25.3 which is incompatible.
requests 2.20.0 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.3 which is incompatible.
awscli 1.16.225 has requirement botocore==1.12.215, but you'll have botocore 1.12.225 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
sagemaker 1.38.0 has requirement urllib3<1.25,>=1.21, but you'll have urllib3 1.25.3 which is incompatible.
requests 2.20.0 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.3 which is incompatible.
awscli 1.16.225 has requirement botocore==1.12.215, but you'll have botocore 1.12.225 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
class BuyerIntentImputer(object):
    @staticmethod
    def stat_based_imputer(df, strategy='median'):
        imp = SimpleImputer(missing_values=np.nan, strategy = strategy)
        return pd.DataFrame(imp.fit_transform(df), columns=df.columns).astype(df.dtypes.to_dict())      
    @staticmethod
    def knn_imputer(df, n_neighbors=30, weights="uniform" ):
        imp = KNNImputer(n_neighbors=n_neighbors, weights=weights)
        return pd.DataFrame(imp.fit_transform(df), columns=df.columns).astype(df.dtypes.to_dict())  
    @staticmethod
    def rforest_imputer(df):
        imp = MissForest()
        return pd.DataFrame(imp.fit_transform(df), columns=df.columns).astype(df.dtypes.to_dict())  

In [13]:
class BuyerIntentDataSetBalancer(object):
    @staticmethod
    #Ballanced the number of samples on each class to median of all classes
    def under_sampling(df, column_label_name):
        res = Counter(df[column_label_name])
        vals=[]
        for k,v in res.items():
           vals.append(v)
        #print(vals)
        m_thresh = int(statistics.median(vals))
        #print(m_thresh)
        df_new = pd.DataFrame()
        for k,v in res.items():
            df_new = pd.concat([df_new, df[df[column_label_name]==k].sample(n=m_thresh, replace=True)])         
        return df_new      
    @staticmethod
    def cluster_abundant_label(df, column_label_name):
        pass

In [14]:
class BuyerIntentStageRuleEngine(object):
    
    @staticmethod
    def rule_3_stages(df, 
                      interval, 
                      weigths = [0.33, 0.33, 1.0-0.66], 
                      rule_name = 'consumer_score'):
        stage=[]
        if rule_name == 'consumer_score':
            for index, row in df.iterrows():
                if row['consumer_score_probablity_to_transact'] <interval[0]:
                    stage.append(0) # Casual Searcher
                elif row['consumer_score_probablity_to_transact'] >=interval[0] and row['consumer_score_probablity_to_transact'] < interval[1]:
                    stage.append(1) # Active Searcher
                else:
                    stage.append(2) # Ready to Transact
        elif rule_name == 'listings_leads_consumer_score':
            total_saved_listings_interval = interval['total_saved_listings']
            total_leads_submitted_interval = interval['total_leads_submitted']
            consumer_score_interval = interval['consumer_score_probablity_to_transact']
            for index, row in df.iterrows():
                if row['total_saved_listings'] < total_saved_listings_interval[0]:
                    if row['total_leads_submitted'] < total_leads_submitted_interval[0]:
                        if row['consumer_score_probablity_to_transact']<consumer_score_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,0,0, weigths)) #Casual Searcher 
                        elif row['consumer_score_probablity_to_transact']>=consumer_score_interval[0] and row['consumer_score_probablity_to_transact']<consumer_score_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,0,1, weigths)) #Casual Searcher
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,0,2, weigths)) #Casual Searcher
                    elif row['total_leads_submitted']>=total_leads_submitted_interval[0] and row['total_leads_submitted']<total_leads_submitted_interval[1]:
                        if row['consumer_score_probablity_to_transact']<consumer_score_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,1,0, weigths)) #Casual Searcher
                        elif row['consumer_score_probablity_to_transact']>=consumer_score_interval[0] and row['consumer_score_probablity_to_transact']<consumer_score_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,1,1, weigths)) #Active Searcher
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,1,2, weigths)) #Weigthed Votes
                    else:
                        if row['consumer_score_probablity_to_transact']<consumer_score_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,2,0, weigths)) #Casual Searcher
                        elif row['consumer_score_probablity_to_transact']>=consumer_score_interval[0] and row['consumer_score_probablity_to_transact']<consumer_score_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,2,1, weigths)) #Weigthed Votes
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,2,2, weigths)) #Ready to Transact
                elif row['total_saved_listings']>=total_saved_listings_interval[0] and row['total_saved_listings']<total_saved_listings_interval[1]:
                    if row['total_leads_submitted']<total_leads_submitted_interval[0]:
                        if row['consumer_score_probablity_to_transact']<consumer_score_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,0,0, weigths)) #Casual Searcher
                        elif row['consumer_score_probablity_to_transact']>=consumer_score_interval[0] and row['consumer_score_probablity_to_transact']<consumer_score_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,0,1, weigths)) #Active Searcher
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,0,2, weigths)) #Wigthed Votes
                    elif row['total_leads_submitted']>=total_leads_submitted_interval[0] and row['total_leads_submitted']<total_leads_submitted_interval[1]:
                        if row['consumer_score_probablity_to_transact']<consumer_score_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,1,0, weigths)) #Active Searcher
                        elif row['consumer_score_probablity_to_transact']>=consumer_score_interval[0] and row['consumer_score_probablity_to_transact']<consumer_score_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,1,1, weigths)) #Active Searcher
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,1,2, weigths)) #Active Searchers
                    else:
                        if row['consumer_score_probablity_to_transact']<consumer_score_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,2,0, weigths)) #Weigthed Votes
                        elif row['consumer_score_probablity_to_transact']>=consumer_score_interval[0] and row['consumer_score_probablity_to_transact']<consumer_score_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,2,1, weigths)) #Active Searchers
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,2,2, weigths)) #Ready to Transact
                else:
                    if row['total_leads_submitted']<total_leads_submitted_interval[0]:
                        if row['consumer_score_probablity_to_transact']<consumer_score_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,0,0, weigths)) #Casual Searcher
                        elif row['consumer_score_probablity_to_transact']>=consumer_score_interval[0] and row['consumer_score_probablity_to_transact']<consumer_score_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,0,1, weigths)) #Wigthed Votes
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,0,2, weigths)) #Ready to Transact
                    elif row['total_leads_submitted']>=total_leads_submitted_interval[0] and row['total_leads_submitted']<total_leads_submitted_interval[1]:
                        if row['consumer_score_probablity_to_transact']<consumer_score_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,1,0, weigths)) #Casual Searcher
                        elif row['consumer_score_probablity_to_transact']>=consumer_score_interval[0] and row['consumer_score_probablity_to_transact']<consumer_score_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,1,1, weigths)) #Active Searcher
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,1,2, weigths)) #Ready to Transact
                    else:
                        if row['consumer_score_probablity_to_transact']<consumer_score_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,2,0, weigths)) #Ready to Transact
                        elif row['consumer_score_probablity_to_transact']>=consumer_score_interval[0] and row['consumer_score_probablity_to_transact']<consumer_score_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,2,1, weigths)) #Ready to Transact
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,2,2, weigths)) #Ready to Transact                        
        elif rule_name == 'searchers_listings_leads':
            total_saved_listings_interval = interval['total_saved_listings']
            total_leads_submitted_interval = interval['total_leads_submitted']
            total_searches_interval = interval['total_searches']
            for index, row in df.iterrows():
                if row['total_saved_listings'] < total_saved_listings_interval[0]:
                    if row['total_leads_submitted'] < total_leads_submitted_interval[0]:
                        if row['total_searches']<total_searches_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,0,0, weigths)) #Casual Searcher 
                        elif row['total_searches']>=total_searches_interval[0] and row['total_searches']<total_searches_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,0,1, weigths)) #Casual Searcher
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,0,2, weigths)) #Casual Searcher
                    elif row['total_leads_submitted']>=total_leads_submitted_interval[0] and row['total_leads_submitted']<total_leads_submitted_interval[1]:
                        if row['total_searches']<total_searches_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,1,0, weigths)) #Casual Searcher
                        elif row['total_searches']>=total_searches_interval[0] and row['total_searches']<total_searches_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,1,1, weigths)) #Active Searcher
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,1,2, weigths)) #Weigthed Votes
                    else:
                        if row['total_searches']<total_searches_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,2,0, weigths)) #Casual Searcher
                        elif row['total_searches']>=total_searches_interval[0] and row['total_searches']<total_searches_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,2,1, weigths)) #Weigthed Votes
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(0,2,2, weigths)) #Ready to Transact
                elif row['total_saved_listings']>=total_saved_listings_interval[0] and row['total_saved_listings']<total_saved_listings_interval[1]:
                    if row['total_leads_submitted']<total_leads_submitted_interval[0]:
                        if row['total_searches']<total_searches_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,0,0, weigths)) #Casual Searcher
                        elif row['total_searches']>=total_searches_interval[0] and row['total_searches']<total_searches_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,0,1, weigths)) #Active Searcher
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,0,2, weigths)) #Wigthed Votes
                    elif row['total_leads_submitted']>=total_leads_submitted_interval[0] and row['total_leads_submitted']<total_leads_submitted_interval[1]:
                        if row['total_searches']<total_searches_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,1,0, weigths)) #Active Searcher
                        elif row['total_searches']>=total_searches_interval[0] and row['total_searches']<total_searches_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,1,1, weigths)) #Active Searcher
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,1,2, weigths)) #Active Searchers
                    else:
                        if row['total_searches']<total_searches_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,2,0, weigths)) #Weigthed Votes
                        elif row['total_searches']>=total_searches_interval[0] and row['total_searches']<total_searches_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,2,1, weigths)) #Active Searchers
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(1,2,2, weigths)) #Ready to Transact
                else:
                    if row['total_leads_submitted']<total_leads_submitted_interval[0]:
                        if row['total_searches']<total_searches_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,0,0, weigths)) #Casual Searcher
                        elif row['total_searches']>=total_searches_interval[0] and row['total_searches']<total_searches_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,0,1, weigths)) #Wigthed Votes
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,0,2, weigths)) #Ready to Transact
                    elif row['total_leads_submitted']>=total_leads_submitted_interval[0] and row['total_leads_submitted']<total_leads_submitted_interval[1]:
                        if row['total_searches']<total_searches_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,1,0, weigths)) #Casual Searcher
                        elif row['total_searches']>=total_searches_interval[0] and row['total_searches']<total_searches_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,1,1, weigths)) #Active Searcher
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,1,2, weigths)) #Ready to Transact
                    else:
                        if row['total_searches']<total_searches_interval[0]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,2,0, weigths)) #Ready to Transact
                        elif row['total_searches']>=total_searches_interval[0] and row['total_searches']<consumer_score_interval[1]:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,2,1, weigths)) #Ready to Transact
                        else:
                            stage.append(BuyerIntentStageRuleEngine.vote_3(2,2,2, weigths)) #Ready to Transact 
        elif rule_name == 'listings_leads':
            total_saved_listings_interval = interval['total_saved_listings']
            total_leads_submitted_interval = interval['total_leads_submitted']
            weigths = [0.3, 1.0-0.3]
            for index, row in df.iterrows():
                if row['total_saved_listings'] < total_saved_listings_interval[0]:
                    if row['total_leads_submitted'] < total_leads_submitted_interval[0]:
                        stage.append(BuyerIntentStageRuleEngine.vote_2(0,0, weigths)) #Casual Searcher|Casual Searcher
                    elif row['total_leads_submitted']>=total_leads_submitted_interval[0] and row['total_leads_submitted']<total_leads_submitted_interval[1]:
                        stage.append(BuyerIntentStageRuleEngine.vote_2(0,1, weigths))
                    else:
                        stage.append(BuyerIntentStageRuleEngine.vote_2(0,2, weigths))
                elif row['total_saved_listings']>=total_saved_listings_interval[0] and row['total_saved_listings']<total_saved_listings_interval[1]:
                    if row['total_leads_submitted']<total_leads_submitted_interval[0]:
                        stage.append(BuyerIntentStageRuleEngine.vote_2(1,0, weigths))
                    elif row['total_leads_submitted']>=total_leads_submitted_interval[0] and row['total_leads_submitted']<total_leads_submitted_interval[1]:
                        stage.append(BuyerIntentStageRuleEngine.vote_2(1,1, weigths)) # Active Searchers
                    else:
                        stage.append(BuyerIntentStageRuleEngine.vote_2(1,2, weigths))
                else:
                    if row['total_leads_submitted']<total_leads_submitted_interval[0]:
                        stage.append(BuyerIntentStageRuleEngine.vote_2(2,0, weigths))
                    elif row['total_leads_submitted']>=total_leads_submitted_interval[0] and row['total_leads_submitted']<total_leads_submitted_interval[1]:
                        stage.append(BuyerIntentStageRuleEngine.vote_2(2,1, weigths)) # 
                    else:
                        stage.append(BuyerIntentStageRuleEngine.vote_2(2,2, weigths)) # Ready to Transact
        return df.assign(buyer_stage = stage ) 
    
    @staticmethod
    def vote_3(a,b,c, weigths):
        if a==b or a==c:
            return a
        elif b==c:
            return b
        else:
            elements = [a, b, c] 
            return choice(elements, p=weigths)
    
    @staticmethod
    def vote_2(a,b, weigths):
        if a==b :
            return a
        else:
            elements = [a, b] 
            return choice(elements, p=weigths)
           
    @staticmethod
    def rule_4_stages(df, interval, rule_name = 'consumer_score'):
        stage=[]
        if rule_name == 'consumer_score':
            for index, row in df.iterrows():
                if row['consumer_score_probablity_to_transact'] <interval[0]:
                    stage.append(0) # Dreamer
                elif row['consumer_score_probablity_to_transact'] >=interval[0] and row['consumer_score_probablity_to_transact'] < interval[1]:
                    stage.append(1) # Casual Searcher
                elif row['consumer_score_probablity_to_transact'] >=interval[1] and row['consumer_score_probablity_to_transact'] < interval[2]:
                    stage.append(2) # Active Searcher
                else:
                    stage.append(3) # Ready to Transact
        elif rule_name == 'lead_consumer_score':
            pass
        return df.assign(buyer_stage = stage )

In [15]:
class BuyerIntentMLModels(object):
    
    @staticmethod
    def run_k_mean(df, up_to_max_cluster_number, title):
        distortions = []
        for i in range(1, up_to_max_cluster_number):
            print('iteration {}'.format(i))
            km = KMeans(
                n_clusters=i, 
                init='random',
                n_init=10, 
                max_iter=300,
                tol=1e-04, 
                random_state=0,
                n_jobs=multiprocessing.cpu_count()-2 
            )
            km.fit(df)
            distortions.append(km.inertia_)

        df_labels = km.predict(df)
        df_centroids = km.cluster_centers_

        # plot number of clusters against distoration to find optimal number of clusters for 
        plt.plot(range(1, up_to_max_cluster_number), distortions, marker='o')
        plt.xlabel('Number of clusters')
        plt.ylabel('Distortion')
        plt.title(title)
        plt.show()

        return (df_labels, df_centroids, distortions )
    
    @staticmethod
    def buyer_intent_stage_classifiers_factory(df, n_s = 10, full_display = True):
        array = df.values
        X = array[:,0:len(df.columns)-1]
        Y = array[:,len(df.columns)-1]
        # prepare configuration for cross validation test harness
        seed = 7
        # prepare models
        models = []
        models.append(('LR', LogisticRegression()))
        models.append(('LDA', LinearDiscriminantAnalysis()))
        models.append(('KNN', KNeighborsClassifier()))
        models.append(('DT', DecisionTreeClassifier()))
        models.append(('NB', GaussianNB()))
        #models.append(('SVM', SVC()))
        models.append(('RF', RandomForestClassifier()))
        models.append(('XGBoost', XGBClassifier()))
        # evaluate each model in turn
        results = []
        names = []
        scoring = 'accuracy'
        best_model={'best_model':'', 'accuracy':0, 'std':0}
        for name, model in models:
            kfold = model_selection.KFold(n_splits = n_s, random_state=seed)
            cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
            results.append(cv_results)
            names.append(name)
            msg = "%s Accuracy: %0.2f (+/- %0.2f)" % (name, cv_results.mean(), cv_results.std())
            if full_display:
                print(msg)
            if best_model['accuracy'] < cv_results.mean():
               best_model['best_model'] = str(name)
               best_model['accuracy'] = cv_results.mean()
               best_model['std'] = cv_results.std()
        # boxplot algorithm comparison
        if full_display:
            fig = plt.figure()
            fig.suptitle('Algorithm Comparison for Buyer Intent Stage Classification')
            ax = fig.add_subplot(111)
            plt.boxplot(results)
            ax.set_xticklabels(names)
            plt.show()
        return best_model